In [ ]:
ls

In [ ]:
cd ..

In [ ]:
!source venv/bin/activate

In [ ]:
from bs4 import BeautifulSoup
import re
from datasets import load_dataset
from tqdm.notebook import tqdm
import pandas as pd
import pickle
import numpy as np

from rag_techniques.relevant_segment_extraction import SegmentExtractionRAG
from research.functions import get_natural_questions_sample, get_ms_marco_sample

In [ ]:
model = SegmentExtractionRAG(collection_name='natural_questions')
df = pd.read_csv('research/data/qa_dataset.csv', index_col=0)

In [ ]:
natural_questions_dataset = {
    "question": [],
    "answer": [],
    "contexts": [],
    "ground_truth": [],
}

In [ ]:
for i, (question, answer) in tqdm(df.loc[:200].iterrows(), total=df.loc[:200].shape[0]):
    try:
        result = model(question)
    except:
        continue
    natural_questions_dataset["question"].append(question)
    natural_questions_dataset["answer"].append(result['answer'])
    natural_questions_dataset["contexts"].append(result['context'])
    if pd.isna(answer):
        answer = 'No answer'
    natural_questions_dataset["ground_truth"].append(answer)   
    if i % 20 == 0:
        with open('research/data/res.pkl', 'wb') as f:
            pickle.dump(natural_questions_dataset, f)

In [ ]:
new_df = pd.DataFrame(natural_questions_dataset)
new_df.to_csv('research/data/relevant_segment_extraction2.csv')

In [ ]:
from src.answer_correctness import AnswerCorrectness

evaluator = AnswerCorrectness()
res = evaluator.get_correctness(natural_questions_dataset['question'], natural_questions_dataset['answer'], natural_questions_dataset['ground_truth'])

In [ ]:
np.mean(res[1])

In [ ]:
with open('research/data/res.pkl', 'wb') as f:
    pickle.dump(natural_questions_dataset, f)

In [ ]:
from src.ragas import Evaluator
ragas = Evaluator()

In [ ]:
e = ragas.eval(natural_questions_dataset)